In [1]:
import numpy as np

# 5.6章Affine_softmaxレイヤの実装

In [ ]:
X = np.random.rand(2) # 入力
w = np.random.rand(2,3) # 重み
B = np.random.rand(3) #バイアス

X.shape # 1行×2列
w.shape # (2行×3列)
B.shape #(1行×3列)
Y = np.dot(X,w) + B


In [ ]:
print(X)
print(w)
print(B)

[0.26569111 0.34219913]
[[0.29082061 0.60015105 0.39132849]
 [0.06553866 0.32253918 0.80189848]]
[0.81765224 0.30516209 0.9446015 ]


In [ ]:
print(np.dot(X,w))

[0.09969572 0.26982742 0.37838146]


In [ ]:
print(Y)

array([0.91734797, 0.57498952, 1.32298296])

## 5.6.2.バッチ版Affineレイヤ

In [5]:
# バッチサイズを指定
batch_size = 2

# 前の層のニューロン数を指定
before = 2

# 次の層のニューロン数を指定
next = 3

In [6]:
# (仮の)入力を作成
X = np.random.rand(batch_size, before)
print(X.shape)

# (仮の)重みを作成
W = np.random.rand(before, next)
print(W.shape)

# (仮の)バイアスを作成
b = np.random.rand(next)
print(b.shape)

(2, 2)
(2, 3)
(3,)


In [7]:
# 重み付き和を計算
Y = np.dot(X, W) + b
print(Y.shape)

(2, 3)


#### 続いて、逆伝播を考える。
- 逆伝播の際には、それぞれのデータの逆伝播の値が、バイアスの要素に集約される必要がある。

In [11]:
# (仮の)逆伝播の入力を作成
dY = np.random.rand(before, next)
print(dY.shape)

(2, 3)


In [12]:
# 順伝播の入力の勾配を計算
dX = np.dot(dY, W.T) 
print(dX.shape)

# 重みの勾配を計算
dW = np.dot(X.T, dY)
print(dW.shape)

# バイアスの勾配を計算
db = np.sum(dY, axis=0)
print(db.shape)

(2, 2)
(2, 3)
(3,)


In [2]:
# Affineレイヤの実装
class Affine:
    # 初期化メソッド
    def __init__(self, W, b):
        # パラメータを保存
        self.W = W # 重み
        self.b = b # バイアス
        
        # 入力と勾配を初期化
        self.x = None  # 順伝播の入力
        self.dW = None # 重みの勾配
        self.db = None # バイアスの勾配
    
    # 順伝播メソッド
    def forward(self, x):
        # 順伝播の入力を保存
        self.x = x
        
        # 重み付き和を計算
        out = np.dot(x, self.W) + self.b
        return out
    
    # 逆伝播メソッド
    def backward(self, dout):
        # 勾配を計算
        dx = np.dot(dout, self.W.T)      # 順伝播の入力の勾配
        self.dW = np.dot(self.x.T, dout) # 重みの勾配
        self.db = np.sum(dout, axis=0)   # バイアスの勾配
        
        # 順伝播の入力の勾配を出力
        return dx

In [8]:
# Affineレイヤのインスタンスを作成
layer = Affine(W, b)
print(layer.x)
print(layer.W.shape)
print(layer.b.shape)

None
(2, 3)
(3,)


In [9]:
# 順伝播を計算
Y = layer.forward(X)
print(Y.shape)

(2, 3)


In [13]:
# 逆伝播を計算
dX = layer.backward(dY)
print(dX.shape)
print(layer.dW.shape)
print(layer.db.shape)

(2, 2)
(2, 3)
(3,)


## 5.6.3 Softmax-with-Lossレイヤ

In [14]:
# Softmax 3.5.2ソフトマックス関数の実装上の注意
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a
  return y

In [15]:
# cross_entropy_error 4.2.4バッチ対応版交差エントロピー誤差の実装
def cross_entropy_error(y, t):
  if y.ndim ==1:
    t = t.reshape(1,t.size)
    y = y.reshape(1,y.size)
  
  batch_size = y.shape[0]
  return -np.sum( t * np.log( y + 1e-7)) / batch_size

In [20]:
# Softmax-with-Lossレイヤの実装
class SoftmaxWithLoss:
    # 初期化メソッド
    def __init__(self):
        # 変数を初期化
        self.loss = None # 交差エントロピー誤差
        self.y = None # ニューラルネットワークの出力
        self.t = None # 教師ラベル
    
    # 順伝播メソッド
    def forward(self, x, t):
        # 教師ラベルを保存
        self.t = t
        
        # ソフトマックス関数による活性化(正規化)
        self.y = softmax(x)
        
        # 交差エントロピー誤差を計算
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    # 逆伝播メソッド
    def backward(self, dout=1):
        # バッチサイズを取得
        batch_size = self.t.shape[0]
        
        # 順伝播の入力の勾配を計算
        dx = (self.y - self.t) / batch_size
        return dx

In [17]:
#バッチサイズを3として実装
# (仮の)入力を作成
A = np.array([
    [1.0, 3.0, 5.0, 7.0, 9.0, 1.5, 3.5, 5.5, 7.5, 9.5], 
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 
    [-10.0, -8.0, -6.0, -4.0,-2.0, 0.0, 2.0, 4.0, 6.0, 8.0]
])
print(A.shape)

# (仮の)教師データを作成
T = np.array([
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])
print(T.shape)

(3, 10)
(3, 10)


In [37]:
# ソフトマックス関数による活性化
Y = softmax(A)
print(np.round(Y, 3))
print(np.sum(Y, axis=1)) # 正規化の確認したのち、交差エントロピー誤差を計算

# 交差エントロピー誤差を計算
L = cross_entropy_error(Y, T)
print(L)

[[0.    0.001 0.005 0.039 0.286 0.    0.001 0.009 0.064 0.472]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.002 0.014 0.105]]
[0.87719998 0.00096067 0.12183935]
5.416781025822352


In [38]:
# バッチサイズを取得
batch_size = T.shape[0]
print(batch_size)

# 逆伝播
dA = (Y - T) / batch_size
print(np.round(dA, 2))
print(dA.shape)

3
[[ 0.    0.    0.    0.01  0.1   0.    0.   -0.33  0.02  0.16]
 [ 0.    0.    0.    0.   -0.33  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   -0.3 ]]
(3, 10)


In [21]:
# Softmax-with-Lossレイヤのインスタンスを作成
layer = SoftmaxWithLoss()

In [39]:
# 順伝播を計算
L = layer.forward(A, T)
print(np.round(layer.y, 2))
print(np.sum(layer.y, axis=1)) # 正規化の確認
print(layer.loss)
print(L)

[[0.   0.   0.01 0.04 0.29 0.   0.   0.01 0.06 0.47]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.11]]
[0.87719998 0.00096067 0.12183935]
5.416781025822352
5.416781025822352


In [40]:
# 逆伝播を計算
dA = layer.backward()
print(np.round(dA, 3))

[[ 0.     0.     0.002  0.013  0.095  0.     0.    -0.33   0.021  0.157]
 [ 0.     0.     0.     0.    -0.333  0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.001  0.005 -0.298]]


## 参考資料

[5.6.3：Softmax-with-Lossレイヤの実装【ゼロつく1のノート(実装)】](https://www.anarchive-beta.com/entry/2020/08/05/180000)

[ニューラルネットがわかりやすくまとめてある。初学者向け](https://tutorials.chainer.org/ja/13_Basics_of_Neural_Networks.html#ref_note5)